<span style='color:darkcyan'><font size="3">_Juan Sebastian Deslarzes_</font></span><br>
<span style='color:darkcyan'><font size="3">_Chloé De Ancos_</font></span><br>
<span style='color:darkcyan'><font size="3">_Anaïs Burrus_</font></span><br>
<span style='color:darkcyan'><font size="3">_Margot Chapot_</font></span><br>
<span style='color:darkcyan'><font size="3">_Eva Sarlin_</font></span><br>

<span style='color:steelblue'><font size="6"> Notebook 3 : use of a unique pipeline for all the features </font></span><br>

In [1]:
import matplotlib.pyplot as plt 
%matplotlib inline
import pandas as pd
import numpy as np
from skimage import io
from IPython import display
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
import seaborn as sns
sns.set(color_codes=True)

Loading BokehJS ...

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from collections import Counter 
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from evolutionary_search import EvolutionaryAlgorithmSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import string

[nltk_data] Downloading package stopwords to /Users/chloe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<span style='color:mediumpurple'><font size="6">__Data Loading__</font></span><br>

In [3]:
sample_submission_url = 'https://raw.githubusercontent.com/esarlin/Challenge-2-CDAW/main/sitc-challenge2-2022/sample_submission.csv'
test_nolabel_url= 'https://raw.githubusercontent.com/esarlin/Challenge-2-CDAW/main/sitc-challenge2-2022/test_nolabel.csv'
train_url = 'https://raw.githubusercontent.com/esarlin/Challenge-2-CDAW/main/sitc-challenge2-2022/train.csv'

sample_submission = pd.read_csv(sample_submission_url)
test_nolabel = pd.read_csv(test_nolabel_url)
df = pd.read_csv(train_url)

<span style='color:mediumpurple'><font size="6">__Feature Selection and Cleaning__</font></span><br>

In [4]:
data_features = df.copy()[["statement", "subject", "speaker","speaker_job","state_info", "party_affiliation"]]
test_features = test_nolabel.copy()[["statement", "subject", "speaker","speaker_job","state_info","party_affiliation"]]
target = df.label.values

<span style='color:steelblue'><font size="4">__Removing Nan values in Speaker job and state features :__</font></span><br>

In [5]:
data_features['speaker_job']= data_features['speaker_job'].fillna('u.s. senator')
test_features['speaker_job']= test_features['speaker_job'].fillna('u.s. senator')

data_features['state_info']= data_features['state_info'].fillna('unknown')
test_features['state_info']= test_features['state_info'].fillna('unknown')

<span style='color:steelblue'><font size="4">__Adding a new feature party_group :__</font></span><br>

We have chosen to add a new feature called "party_group" in which we have separated the feature "party_affiliation" into three groups : 0 which corresponds to republican, 1 to democrats and 2 to others.

In [6]:
data_features['party_group'] = data_features['party_affiliation']
test_features['party_group'] = test_features['party_affiliation']

In [7]:
party = data_features.groupby('party_affiliation').size()
print(f'Names of the party_affiliations of the speakers, and how many of them are affiliated: \n{party}')

party_test = test_features.groupby('party_affiliation').size()
print(f'Names of the party_affiliations of the speakers, and how many of them are affiliated: \n{party_test}')

Names of the party_affiliations of the speakers, and how many of them are affiliated: 
party_affiliation
Moderate                           1
activist                          30
business-leader                    8
columnist                         32
constitution-party                 2
county-commissioner                1
democrat                        2898
democratic-farmer-labor            1
education-official                 2
government-body                    1
green                              3
independent                      130
journalist                        37
labor-leader                       7
liberal-party-canada               1
libertarian                       34
newsmaker                         41
none                            1531
ocean-state-tea-party-action       1
organization                     197
republican                      3947
state-official                    14
talk-show-host                    23
tea-party-member                   8
dtype: 

In [8]:
#Encode new feature 
data_features.loc[data_features["party_group"] == "republican", "party_group"] = 0
data_features.loc[data_features["party_group"] == "democrat", "party_group"] = 1
data_features.loc[data_features["party_group"] == "none", "party_group"] = 2
data_features.loc[data_features["party_group"] == "independent", "party_group"] = 2
data_features.loc[data_features["party_group"] == "journalist", "party_group"] = 2
data_features.loc[data_features["party_group"] == "organization", "party_group"] = 2
data_features.loc[data_features["party_group"] == "talk-show-host", "party_group"] = 2
data_features.loc[data_features["party_group"] == "tea-party-member", "party_group"] = 0
data_features.loc[data_features["party_group"] == "activist", "party_group"] = 2
data_features.loc[data_features["party_group"] == "constitution-party", "party_group"] = 0
data_features.loc[data_features["party_group"] == "libertarian", "party_group"] = 0
data_features.loc[data_features["party_group"] == "state-official", "party_group"] = 2
data_features.loc[data_features["party_group"] == "columnist", "party_group"] = 2
data_features.loc[data_features["party_group"] == "newsmaker", "party_group"] = 2
data_features.loc[data_features["party_group"] == "Moderate", "party_group"] = 2
data_features.loc[data_features["party_group"] == "county-commissioner", "party_group"] = 2
data_features.loc[data_features["party_group"] == "green", "party_group"] = 1
data_features.loc[data_features["party_group"] == "liberal-party-canada", "party_group"] = 0
data_features.loc[data_features["party_group"] == "ocean-state-tea-party-action", "party_group"] = 1
data_features.loc[data_features["party_group"] == "government-body", "party_group"] = 2
data_features.loc[data_features["party_group"] == "democratic-farmer-labor", "party_group"] = 1
data_features.loc[data_features["party_group"] == "business-leader", "party_group"] = 0
data_features.loc[data_features["party_group"] == "education-official", "party_group"] = 0
data_features.loc[data_features["party_group"] == "labor-leader", "party_group"] = 1

test_features.loc[test_features["party_group"] == "republican", "party_group"] = 0
test_features.loc[test_features["party_group"] == "democrat", "party_group"] = 1
test_features.loc[test_features["party_group"] == "none", "party_group"] = 2
test_features.loc[test_features["party_group"] == "independent", "party_group"] = 2
test_features.loc[test_features["party_group"] == "journalist", "party_group"] = 2
test_features.loc[test_features["party_group"] == "organization", "party_group"] = 2
test_features.loc[test_features["party_group"] == "talk-show-host", "party_group"] = 2
test_features.loc[test_features["party_group"] == "tea-party-member", "party_group"] = 0
test_features.loc[test_features["party_group"] == "activist", "party_group"] = 2
test_features.loc[test_features["party_group"] == "constitution-party", "party_group"] = 0
test_features.loc[test_features["party_group"] == "libertarian", "party_group"] = 0
test_features.loc[test_features["party_group"] == "state-official", "party_group"] = 2
test_features.loc[test_features["party_group"] == "columnist", "party_group"] = 2
test_features.loc[test_features["party_group"] == "newsmaker", "party_group"] = 2
test_features.loc[test_features["party_group"] == "Moderate", "party_group"] = 2
test_features.loc[test_features["party_group"] == "county-commissioner", "party_group"] = 2
test_features.loc[test_features["party_group"] == "green", "party_group"] = 1
test_features.loc[test_features["party_group"] == "liberal-party-canada", "party_group"] = 0
test_features.loc[test_features["party_group"] == "ocean-state-tea-party-action", "party_group"] = 1
test_features.loc[test_features["party_group"] == "government-body", "party_group"] = 2
test_features.loc[test_features["party_group"] == "democratic-farmer-labor", "party_group"] = 1
test_features.loc[test_features["party_group"] == "business-leader", "party_group"] = 0
test_features.loc[test_features["party_group"] == "education-official", "party_group"] = 0
test_features.loc[test_features["party_group"] == "labor-leader", "party_group"] = 1

In [9]:
data_features.head()

,statement,subject,speaker,speaker_job,state_info,party_affiliation,party_group
0,China is in the South China Sea and (building)...,"china,foreign-policy,military",donald-trump,President-Elect,New York,republican,0
1,With the resources it takes to execute just ov...,health-care,chris-dodd,U.S. senator,Connecticut,democrat,1
2,The (Wisconsin) governor has proposed tax give...,"corporations,pundits,taxes,abc-news-week",donna-brazile,Political commentator,"Washington, D.C.",democrat,1
3,Says her representation of an ex-boyfriend who...,"candidates-biography,children,ethics,families,...",rebecca-bradley,u.s. senator,unknown,none,2
4,At protests in Wisconsin against proposed coll...,"health-care,labor,state-budget",republican-party-wisconsin,u.s. senator,Wisconsin,republican,0


In [34]:
data_features.isnull().sum()

statement            0
subject              0
speaker              0
speaker_job          0
state_info           0
party_affiliation    0
party_group          0
subjectivityRange    0
dtype: int64

In [10]:
from textblob import TextBlob

data_features["subjectivity"] = data_features["statement"].apply(lambda x: 
                   TextBlob(x).sentiment.subjectivity)
test_features["subjectivity"] = test_features["statement"].apply(lambda x: 
                   TextBlob(x).sentiment.subjectivity)
data_features.head()

,statement,subject,speaker,speaker_job,state_info,party_affiliation,party_group,subjectivity
0,China is in the South China Sea and (building)...,"china,foreign-policy,military",donald-trump,President-Elect,New York,republican,0,0.100000
1,With the resources it takes to execute just ov...,health-care,chris-dodd,U.S. senator,Connecticut,democrat,1,0.000000
2,The (Wisconsin) governor has proposed tax give...,"corporations,pundits,taxes,abc-news-week",donna-brazile,Political commentator,"Washington, D.C.",democrat,1,0.000000
3,Says her representation of an ex-boyfriend who...,"candidates-biography,children,ethics,families,...",rebecca-bradley,u.s. senator,unknown,none,2,0.076923
4,At protests in Wisconsin against proposed coll...,"health-care,labor,state-budget",republican-party-wisconsin,u.s. senator,Wisconsin,republican,0,0.635714


In [11]:
data_features.loc[data_features['subjectivity'] >= 0.8, 'subjectivityRange'] = int(0)
data_features.loc[(data_features['subjectivity'] < 0.8) & (data_features['subjectivity'] >= 0.6), 'subjectivityRange'] = int(1)
data_features.loc[(data_features['subjectivity'] < 0.6) & (data_features['subjectivity'] >= 0.4), 'subjectivityRange'] = int(2)
data_features.loc[(data_features['subjectivity'] < 0.4) & (data_features['subjectivity'] >= 0.2), 'subjectivityRange'] = int(3)
data_features.loc[data_features['subjectivity'] < 0.2, 'subjectivityRange'] = int(4)

data_features.drop('subjectivity', axis=1, inplace=True)
data_features.head()

,statement,subject,speaker,speaker_job,state_info,party_affiliation,party_group,subjectivityRange
0,China is in the South China Sea and (building)...,"china,foreign-policy,military",donald-trump,President-Elect,New York,republican,0,4.0
1,With the resources it takes to execute just ov...,health-care,chris-dodd,U.S. senator,Connecticut,democrat,1,4.0
2,The (Wisconsin) governor has proposed tax give...,"corporations,pundits,taxes,abc-news-week",donna-brazile,Political commentator,"Washington, D.C.",democrat,1,4.0
3,Says her representation of an ex-boyfriend who...,"candidates-biography,children,ethics,families,...",rebecca-bradley,u.s. senator,unknown,none,2,4.0
4,At protests in Wisconsin against proposed coll...,"health-care,labor,state-budget",republican-party-wisconsin,u.s. senator,Wisconsin,republican,0,1.0


In [12]:
test_features.loc[test_features['subjectivity'] >= 0.8, 'subjectivityRange'] = int(0)
test_features.loc[(test_features['subjectivity'] < 0.8) & (test_features['subjectivity'] >= 0.6), 'subjectivityRange'] = int(1)
test_features.loc[(test_features['subjectivity'] < 0.6) & (test_features['subjectivity'] >= 0.4), 'subjectivityRange'] = int(2)
test_features.loc[(test_features['subjectivity'] < 0.4) & (test_features['subjectivity'] >= 0.2), 'subjectivityRange'] = int(3)
test_features.loc[test_features['subjectivity'] < 0.2, 'subjectivityRange'] = int(4)

test_features.drop('subjectivity', axis=1, inplace=True)
test_features.head()

,statement,subject,speaker,speaker_job,state_info,party_affiliation,party_group,subjectivityRange
0,Five members of [the Common Cause Georgia] boa...,"campaign-finance,ethics,government-regulation",kasim-reed,u.s. senator,unknown,democrat,1,2.0
1,Theres no negative advertising in my campaign ...,elections,bill-mccollum,u.s. senator,Florida,republican,0,2.0
2,Leticia Van de Putte voted to give illegal imm...,"health-care,immigration,public-health",dan-patrick,Lieutenant governor-elect,Texas,republican,0,1.0
3,Fiorinas plan would mean slashing Social Secur...,"federal-budget,medicare,social-security",barbara-boxer,U.S. Senator,California,democrat,1,3.0
4,"By the end of his first term, President Obama ...","federal-budget,new-hampshire-2012",mitt-romney,Former governor,Massachusetts,republican,0,4.0


<span style='color:mediumpurple'><font size="6">__Natural Language Processing__</font></span><br>


<span style='color:steelblue'><font size="4">__Defining functions for string processing__</font></span><br>

In our transformers will perform the following tasks:
- split the sentences 
- split the words
- remove stop words 
- remove the punctuation
- modify capital letters in lowercases
- remove morphological affixes with stemmers, leaving only the word stem 
- we also perform POS tagging which assigns a grammatical category to a word
- transform the text into numerical feature vectors using vectorizer 

In [13]:
# Generic Transformer 
class GenericTransformer(BaseEstimator, TransformerMixin):

    def transform(self, X, y=None):
        return do_something_to(X, self.vars)  # where the actual feature extraction happens

    def fit(self, X, y=None):
        return self  # used if the feature requires training, for example, clustering
    
    
    
class LexicalStats (BaseEstimator, TransformerMixin):
    """Extract lexical features from each document"""
    
    def number_sentences(self, doc):
        sentences = sent_tokenize(doc, language='english')
        return len(sentences)

    def fit(self, x, y=None):
        return self

    def transform(self, docs):
        return [{'length': len(doc),
                 'num_sentences': self.number_sentences(doc)}
                for doc in docs]
    
def custom_tokenizer(words):
    """Preprocessing tokens as seen in the lexical notebook"""
    tokens = word_tokenize(words.lower())
    porter = PorterStemmer()
    lemmas = [porter.stem(t) for t in tokens]
    stoplist = stopwords.words('english')
    lemmas_clean = [w for w in lemmas if w not in stoplist]
    punctuation = set(string.punctuation)
    lemmas_punct = [w for w in lemmas_clean if  w not in punctuation]
    return lemmas_punct

class PosStats(BaseEstimator, TransformerMixin):
    """Obtain number of tokens with POS categories"""

    def stats(self, doc):
        tokens = custom_tokenizer(doc)
        tagged = pos_tag(tokens, tagset='universal')
        counts = Counter(tag for word,tag in tagged)
        total = sum(counts.values())
        #copy tags so that we return always the same number of features
        pos_features = {'NOUN': 0, 'ADJ': 0, 'VERB': 0, 'ADV': 0, 'CONJ': 0, 
                        'ADP': 0, 'PRON':0, 'NUM': 0}
        
        pos_dic = dict((tag, float(count)/total) for tag,count in counts.items())
        for k in pos_dic:
            if k in pos_features:
                pos_features[k] = pos_dic[k]
        return pos_features
    
    def transform(self, docs, y=None):
        return [self.stats(doc) for doc in docs]
    
    def fit(self, docs, y=None):
        """Returns `self` unless something different happens in train and test"""
        return self
    
ngrams_featurizer = Pipeline([
  ('count_vectorizer',  CountVectorizer(ngram_range = (1, 3), encoding = 'ISO-8859-1', 
                                        tokenizer=custom_tokenizer)),
  ('tfidf_transformer', TfidfTransformer())
])

In [14]:
transformer = FeatureUnion([
                    ('lexical_stats', Pipeline([
                                ('stats', LexicalStats()),
                                ('vectors', DictVectorizer())
                            ])),
                    ('words', TfidfVectorizer(tokenizer=custom_tokenizer)),
                    ('ngrams', ngrams_featurizer),
                    ('pos_stats', Pipeline([
                                ('pos_stats', PosStats()),
                                ('vectors', DictVectorizer())
                            ])),
                    ('lda', Pipeline([ 
                                ('count', CountVectorizer(tokenizer=custom_tokenizer)),
                                ('lda',  LatentDirichletAllocation(n_components=4, max_iter=5,
                                                       learning_method='online', 
                                                       learning_offset=50.,
                                                       random_state=0))
                            ])),
                ])

In [74]:
class IdentityTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass 
    
    def fit(self, input_array, y=None):
        return self
    
    def transform(self, input_array, y=None):
        return input_array.reshape(-1, 1)

In [75]:
from sklearn.preprocessing import OneHotEncoder

identityTransformer = FeatureUnion([
                           ('identity', Pipeline([
                               ('id', IdentityTransformer()),  
                               ('scaler', StandardScaler())
                        ]))
                ])

<span style='color:steelblue'><font size="4">__Applying the processing techniques on all the features using ColumnTransformer :__</font></span><br>


__ColumnTransformer__ applies transformers to the columns of our dataset. This will allow to transform each acolumn separytely and then to concatenate them to form a single feature space. 

In [92]:
from sklearn.preprocessing import OrdinalEncoder
col_preprocessor = ColumnTransformer(
        [
            ('title_glove', transformer, 'statement'),
            ('description_glove', transformer, 'subject'),
            ('speaker_glove', transformer, 'speaker'),
            ('speakerjob_glove', transformer, 'speaker_job'),
            ('stateinfo_glove', transformer, 'state_info'),
            ('party_glove', transformer, 'party_affiliation')
            
],
        remainder='passthrough',
        n_jobs=1
        )

<span style='color:mediumpurple'><font size="6">__Machine Learning algorithm__</font></span><br>

In this section we are going to use pipelines to extract our features. 
Moreover we are going to implement different __pipelines__ that use different classifier in order to deduct which one is the best. In order to achieve this, we will create 2 matrix with the output of the pipelines. One will have the labels predicted with the test features. We will use this first one for the precitions. The second matrix will be composed of the lables predicted with the train features. This matrix will be used as a training set to perform a second classification by the 3 algorithm into the __Voting classifier__. 

<span style='color:steelblue'><font size="4">__Classifier insertion into the pipeline:__</font></span><br>

In [93]:
pipeline1 = Pipeline([
    ('col_preprocessor', col_preprocessor), 
    ('classifier', MultinomialNB(alpha=.01))
])

<span style='color:steelblue'><font size="4">__Training :__</font></span><br>

In [94]:
pipeline1.fit(data_features, target)
y_p1 = pipeline1.predict(test_features)
y_1 = pipeline1.predict(data_features)

Now we will create 3 other pipelines but using 3 different classifiers: Decision tree and kNN.

In [95]:
pipeline2 = Pipeline([
    ('col_preprocessor', col_preprocessor), 
    ('classifier', DecisionTreeClassifier(random_state= 22, max_leaf_nodes = 10, max_depth =3))
     ])

pipeline2.fit(data_features, target)
y_p2 = pipeline2.predict(test_features)
y_2 = pipeline2.predict(data_features)

In [96]:
pipeline3 = Pipeline([
    ('col_preprocessor', col_preprocessor), 
    ('classifier', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                                        metric_params=None, n_jobs=None, n_neighbors=5, 
                                        p=2, weights='uniform'))
    ])

pipeline3.fit(data_features, target)
y_p3 = pipeline3.predict(test_features)
y_3 = pipeline3.predict(data_features)

Here we are going to create the matrix with the outputs of the 3 pipelines. 

In [97]:
y_pt = np.array([y_p1,y_p2,y_p3])
y_p = np.transpose(y_pt)
print(y_p)
y_p.shape

[[1 0 1]
 [1 1 1]
 [1 1 0]
 ...
 [0 1 1]
 [0 1 0]
 [1 1 1]]


(3836, 3)

y_p will be used for the predictions later. 

In [98]:
y_tt = np.array([y_1,y_2,y_3])
y_t = np.transpose(y_tt)
print(y_t)
y_t.shape

[[1 1 1]
 [0 0 0]
 [0 0 1]
 ...
 [0 1 1]
 [1 1 1]
 [0 1 0]]


(8950, 3)

y_t will be used as a training set later. 

<span style='color:mediumpurple'><font size="6">__Ensemble Algorithms__</font></span><br>

<span style='color:steelblue'><font size="4">MultinomialNB</font></span><br>

In [121]:
MNB_clf = MultinomialNB(alpha=0.0001)
MNB_clf.fit(y_t, target)
y_predicted = MNB_clf.predict(y_p)

<span style='color:steelblue'><font size="4">Decision Tree</font></span><br>

In [122]:
DT_clf = DecisionTreeClassifier(random_state=7, criterion='entropy', min_samples_split=5, max_depth=1)
DT_clf.fit(y_t,target)
y_predicted = DT_clf.predict(y_p)

<span style='color:steelblue'><font size="4">kNN</font></span><br>

In [123]:
kNN_clf = KNeighborsClassifier(n_neighbors= 7, weights='uniform', algorithm='brute')
kNN_clf.fit(y_t,target)
y_predicted = kNN_clf.predict(y_p)

<span style='color:steelblue'><font size="4">__Voting classifier__</font></span><br>

In [124]:
VotingModel = VotingClassifier(estimators=[('MultinomialNB', MNB_clf),('DecisionTree', DT_clf), ('kNN', kNN_clf)])
VotingModel.fit(y_t, target)

VotingClassifier(estimators=[('MultinomialNB', MultinomialNB(alpha=0.0001)),
                             ('DecisionTree',
                              DecisionTreeClassifier(criterion='entropy',
                                                     max_depth=1,
                                                     min_samples_split=5,
                                                     random_state=7)),
                             ('kNN',
                              KNeighborsClassifier(algorithm='brute',
                                                   n_neighbors=7))])

<span style='color:mediumpurple'><font size="6">__Hyperparameter tuning using Genetic Algorithm__</font></span><br>

<span style='color:steelblue'><font size="4">MultinomialNB</font></span><br>

In [114]:
param_grid_MNB = [{'alpha': np.arange(0.0, 0.8),    
                    }]

In [115]:
MNB_GA = EvolutionaryAlgorithmSearchCV(estimator=MultinomialNB(),
                                       params=param_grid_MNB,
                                       scoring="accuracy",
                                       cv=StratifiedKFold(n_splits=2),
                                       verbose=True,
                                       population_size=50,
                                       gene_mutation_prob=0.10,
                                       tournament_size=20,
                                       generations_number=10)
%time MNB_GA.fit(y_t, target)

Types [2] and maxint [0] detected
--- Evolve in 1 possible combinations ---
gen	nevals	avg     	min     	max     	std        
0  	50    	0.781788	0.781788	0.781788	1.11022e-16
1  	32    	0.781788	0.781788	0.781788	1.11022e-16
2  	35    	0.781788	0.781788	0.781788	1.11022e-16
3  	31    	0.781788	0.781788	0.781788	1.11022e-16
4  	23    	0.781788	0.781788	0.781788	1.11022e-16
5  	33    	0.781788	0.781788	0.781788	1.11022e-16
6  	29    	0.781788	0.781788	0.781788	1.11022e-16
7  	36    	0.781788	0.781788	0.781788	1.11022e-16
8  	27    	0.781788	0.781788	0.781788	1.11022e-16
9  	25    	0.781788	0.781788	0.781788	1.11022e-16
10 	30    	0.781788	0.781788	0.781788	1.11022e-16
Best individual is: {'alpha': 0.0}
with fitness: 0.7817877094972067
CPU times: user 56.9 ms, sys: 5.25 ms, total: 62.2 ms
Wall time: 60.1 ms


/Users/chloe/opt/anaconda3/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/chloe/opt/anaconda3/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/chloe/opt/anaconda3/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/Users/chloe/opt/anaconda3/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/Users/chloe/opt/a

<span style='color:steelblue'><font size="4">Decision Tree</font></span><br>

In [116]:
param_grid_DT = [{'random_state' : np.arange(0,10),
                  'criterion' : ['gini', 'entropy'],
                  'min_samples_split' : np.arange(2,6), 
                  'max_depth' : np.arange(1,3)       
                    }]

In [117]:
DT_GA = EvolutionaryAlgorithmSearchCV(estimator=DecisionTreeClassifier(),
                                       params=param_grid_DT,
                                       scoring="accuracy",
                                       cv=StratifiedKFold(n_splits=2),
                                       verbose=True,
                                       population_size=50,
                                       gene_mutation_prob=0.10,
                                       tournament_size=20,
                                       generations_number=10)
%time DT_GA.fit(y_t, target)

/Users/chloe/opt/anaconda3/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/chloe/opt/anaconda3/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Types [1, 1, 1, 1] and maxint [9, 1, 3, 1] detected
--- Evolve in 160 possible combinations ---
gen	nevals	avg     	min     	max     	std
0  	50    	0.994078	0.994078	0.994078	0  
1  	33    	0.994078	0.994078	0.994078	0  
2  	29    	0.994078	0.994078	0.994078	0  
3  	23    	0.994078	0.994078	0.994078	0  
4  	36    	0.994078	0.994078	0.994078	0  
5  	29    	0.994078	0.994078	0.994078	0  
6  	24    	0.994078	0.994078	0.994078	0  
7  	32    	0.994078	0.994078	0.994078	0  
8  	31    	0.994078	0.994078	0.994078	0  
9  	24    	0.994078	0.994078	0.994078	0  
10 	32    	0.994078	0.994078	0.994078	0  
Best individual is: {'random_state': 7, 'criterion': 'entropy', 'min_samples_split': 5, 'max_depth': 1}
with fitness: 0.9940782122905027
CPU times: user 753 ms, sys: 29.2 ms, total: 782 ms
Wall time: 785 ms


<span style='color:steelblue'><font size="4">kNN</font></span><br>

In [118]:
param_grid_kNN = [{'n_neighbors': np.arange(4, 8),
                     'weights': ['uniform', 'distance'],
                    'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute']           
                    }]

In [119]:
kNN_GA = EvolutionaryAlgorithmSearchCV(estimator=KNeighborsClassifier(),
                                       params=param_grid_kNN,
                                       scoring="accuracy",
                                       cv=StratifiedKFold(n_splits=2),
                                       verbose=True,
                                       population_size=50,
                                       gene_mutation_prob=0.10,
                                       tournament_size=20,
                                       generations_number=10)
%time kNN_GA.fit(y_t, target)

Types [1, 1, 1] and maxint [3, 1, 3] detected
--- Evolve in 32 possible combinations ---


/Users/chloe/opt/anaconda3/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/chloe/opt/anaconda3/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg     	min     	max     	std
0  	50    	0.994078	0.994078	0.994078	0  
1  	25    	0.994078	0.994078	0.994078	0  
2  	40    	0.994078	0.994078	0.994078	0  
3  	26    	0.994078	0.994078	0.994078	0  
4  	31    	0.994078	0.994078	0.994078	0  
5  	35    	0.994078	0.994078	0.994078	0  
6  	27    	0.994078	0.994078	0.994078	0  
7  	27    	0.994078	0.994078	0.994078	0  
8  	32    	0.994078	0.994078	0.994078	0  
9  	34    	0.994078	0.994078	0.994078	0  
10 	28    	0.994078	0.994078	0.994078	0  
Best individual is: {'n_neighbors': 7, 'weights': 'uniform', 'algorithm': 'brute'}
with fitness: 0.9940782122905027
CPU times: user 21.2 s, sys: 2.63 s, total: 23.8 s
Wall time: 13.7 s


<span style='color:mediumpurple'><font size="6">__Prediction__</font></span><br>

In [125]:
y_predicted_voting = VotingModel.predict(y_p)

<div class="alert alert-block alert-info">
    <span style='color:darkblue'><font size="4"><b>CONCLUSION ALGORITHMS</b></font></span><br><br> <span style='color:black'> After several machine algorithms, we can conclude that the multinomial gives us the best accuracy on Kaggle which is 58%.
    Once we added the VotingClassifier with kNN, Decision Tree and MultinomialNB the accuracy on kaggle didn't change meaning that the multinomial algorithm is the best one. </span>
</div>

<span style='color:mediumpurple'><font size="6">__Submission__</font></span><br>

In [126]:
id = test_nolabel['id']
dfsubmission = pd.DataFrame({'id': id, 'label': y_predicted_voting}, columns=["id", "label"])
dfsubmission.to_csv('submission0.csv', index=False, header=True)

<div class="alert alert-block alert-info">
    <span style='color:purple'><font size="4"><b>CONCLUSION GENERALE</b></font></span><br><br> <span style='color:black'> This methods, by applying complete natural language processing to each features in parallel using a pipeline gives us the best accuracy of 58.2%, using a multinomial classification algorithm. </span>
</div>